<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Feature_Engineering/Feature_Engineering_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 普通はさ、色んなエンコーディングとか特徴量生成をしてたら、数100, 数1000とか出てくるよね？
* これには2つの問題があります。
    * 1つ目は、特徴量が多ければ多い程、
        * **トレーニングセットとバリデーションセットにオーバーフィットする可能性が高く**なります
        * これは、新しいデータへの一般化においてモデルの性能を低下させる原因となります
    * 2つ目は、特徴量が多ければ多い程、
        * モデルを訓練してハイパーパラメータを最適化するのに時間がかかります。
        * また、ユーザ向けの製品を構築する場合、推論を可能な限り高速化したいと思うでしょう
        * 少ない特徴量を使用すると、**予測性能を犠牲にして推論を高速化する**ことが出来ます。
* これらの問題を解決するために、
* **特徴選択テクニックを使用して、モデルに最も有益な特徴を維持することが出来ます。**

In [1]:
# %matplotlib inline

import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

ks = pd.read_csv('/content/ks-projects-201801.csv',
                 parse_dates=['deadline', 'launched'])

# Drop live projects
ks = ks.query('state != "live"')

# Add outcome column, "successful" == 1, others are 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))

# Timestamp features
ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

# Label encoding
cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()
encoded = ks[cat_features].apply(encoder.fit_transform)

data_cols = ['goal', 'hour', 'day', 'month', 'year', 'outcome']
baseline_data = ks[data_cols].join(encoded)

cat_features = ['category', 'currency', 'country']
interactions = pd.DataFrame(index=ks.index)
for col1, col2 in itertools.combinations(cat_features, 2):
    new_col_name = '_'.join([col1, col2])
    # Convert to strings and combine
    new_values = ks[col1].map(str) + "_" + ks[col2].map(str)
    label_enc = LabelEncoder()
    interactions[new_col_name] = label_enc.fit_transform(new_values)
baseline_data = baseline_data.join(interactions)

launched = pd.Series(ks.index, index=ks.launched, name="count_7_days").sort_index()
count_7_days = launched.rolling('7d').count() - 1
count_7_days.index = launched.values
count_7_days = count_7_days.reindex(ks.index)

baseline_data = baseline_data.join(count_7_days)

def time_since_last_project(series):
    # Return the time in hours
    return series.diff().dt.total_seconds() / 3600.

df = ks[['category', 'launched']].sort_values('launched')
timedeltas = df.groupby('category').transform(time_since_last_project)
timedeltas = timedeltas.fillna(timedeltas.max())

baseline_data = baseline_data.join(timedeltas.rename({'launched': 'time_since_last_project'}, axis=1))

def get_data_splits(dataframe, valid_fraction=0.1):
    valid_fraction = 0.1
    valid_size = int(len(dataframe) * valid_fraction)

    train = dataframe[:-valid_size * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_size * 2:-valid_size]
    test = dataframe[-valid_size:]
    
    return train, valid, test

def train_model(train, valid):
    feature_cols = train.columns.drop('outcome')

    dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

    param = {'num_leaves': 64, 'objective': 'binary', 
             'metric': 'auc', 'seed': 7}
    print("Training model!")
    bst = lgb.train(param, dtrain, num_boost_round=1000, valid_sets=[dvalid], 
                    early_stopping_rounds=10, verbose_eval=False)

    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['outcome'], valid_pred)
    print(f"Validation AUC score: {valid_score:.4f}")
    return bst

# Univariate Feature Selection 一変量特徴の選択
* 最もシンプルで高速な方法は、一変量統計検定に基づいています。
    * 各特徴について、χ^2 とか ANOVAのような統計的検定を使って、
    * ターゲットがどれだけ特徴に強く依存しているかを測定できるよ！
* scikit-learnの特徴選択モジュールから、
    * sklearn.feature_selection.SelectKBest を使え
    * いくつかのスコアリング関数を与えられたK個の最良の特徴を返します。
    * 我々の分類問題では、このモジュールは3つのスコアリング関数を提供する
        * χ^2, ANOVA F-Value、そして相互情報スコア(mutual information score)を使います.
    * F- 値は**特徴変数とターゲット間の線形依存性**を測定
        * これは、間の関係が日線形である場合、スコアは特徴変数とターゲットの関係を過小評価する可能性があることが出来ます。
        * 相互情報スコアは、ノンパラメトリックであるため、**非線形関係**をとらえることが出来ます
* SelectKBestでは、スコアリング関数のスコアに基づいて、保持する特徴の数を定義します。
    * .fit_transform(features, target)を使うと、選択された特徴量のみの配列が得られます


In [2]:
baseline_data

,goal,hour,day,month,year,outcome,category,currency,country,category_currency,category_country,currency_country,count_7_days,time_since_last_project
0,1000.0,12,11,8,2015,0,108,5,9,794,1119,17,160.0,786.025000
1,30000.0,4,2,9,2017,0,93,13,22,691,964,29,98.0,59.598333
2,45000.0,0,12,1,2013,0,93,13,22,691,964,29,81.0,359.383333
3,5000.0,3,17,3,2012,0,90,13,22,675,947,29,103.0,1.293889
4,19500.0,8,4,7,2015,0,55,13,22,423,591,29,145.0,131.568333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40908,1000.0,18,18,7,2014,1,65,13,22,499,704,29,326.0,87.986389
40909,1500.0,17,3,5,2012,1,55,13,22,423,591,29,102.0,12.569444
40910,30000.0,22,1,8,2014,1,40,13,22,314,443,29,253.0,73.143611
40911,230000.0,6,10,3,2012,0,39,13,22,305,430,29,93.0,5.880000


In [5]:
from sklearn.feature_selection import SelectKBest, f_classif
# セット？
feature_cols = baseline_data.columns.drop('outcome')
selector = SelectKBest(f_classif, k = 5)
X_new = selector.fit_transform(baseline_data[feature_cols], baseline_data['outcome'])
X_new

array([[2015.,    5.,    9.,   17.,  160.],
       [2017.,   13.,   22.,   29.,   98.],
       [2013.,   13.,   22.,   29.,   81.],
       ...,
       [2014.,   13.,   22.,   29.,  253.],
       [2012.,   13.,   22.,   29.,   93.],
       [2015.,    4.,   17.,   16.,   76.]])

## ここで間違えたことをした
* 統計的な検定は、すべてのデータを使って計算されます。
* これは検証セットとテストセットからの情報が、我々が保持する特徴に影響を与え、
* 漏れ(leakage)の原因を導入する可能性があることを意味します。
* つまり、**学習のセットだけを使って特徴を選択すべき**だということです。

In [6]:
feature_cols = baseline_data.columns.drop('outcome')
train, valid, _ = get_data_splits(baseline_data)

selector = SelectKBest(f_classif, k = 5)
X_new = selector.fit_transform(train[feature_cols], train['outcome'])
X_new

array([[2015.,    5.,    9.,   17.,  160.],
       [2017.,   13.,   22.,   29.,   98.],
       [2013.,   13.,   22.,   29.,   81.],
       ...,
       [2012.,   13.,   22.,   29.,   96.],
       [2015.,   13.,   22.,   29.,  153.],
       [2013.,   13.,   22.,   29.,   84.]])

* 選択された特徴量が、データセット全体を使用した時とは異なることに気が付くはず
* これで選択された特徴量が得られましたが、それはトレーニング・セットの特徴量だけです。
* 検証セットとテストセットから拒絶された特徴を削除するには、
* データセットの度の列がSelectKBestで保持されてるかを把握する必要があります。
* これを行うには、**.inverse_transform**を使用して、元のでーたの形状を持つ配列を取得しています。

In [7]:
selected_features = pd.DataFrame(selector.inverse_transform(X_new),
                                 index = train.index,
                                 columns = feature_cols)
selected_features.head()

,goal,hour,day,month,year,category,currency,country,category_currency,category_country,currency_country,count_7_days,time_since_last_project
0,0.0,0.0,0.0,0.0,2015.0,0.0,5.0,9.0,0.0,0.0,17.0,160.0,0.0
1,0.0,0.0,0.0,0.0,2017.0,0.0,13.0,22.0,0.0,0.0,29.0,98.0,0.0
2,0.0,0.0,0.0,0.0,2013.0,0.0,13.0,22.0,0.0,0.0,29.0,81.0,0.0
3,0.0,0.0,0.0,0.0,2012.0,0.0,13.0,22.0,0.0,0.0,29.0,103.0,0.0
4,0.0,0.0,0.0,0.0,2015.0,0.0,13.0,22.0,0.0,0.0,29.0,145.0,0.0


## 分散が0ではない特徴量を選択することで、選択された列を見つけることが出来る
* ドロップされた列はすべてゼロで埋められています

In [8]:
selected_columns = selected_features.columns[selected_features.var() != 0]

valid[selected_columns].head()

,year,currency,country,currency_country,count_7_days
32739,2016,11,20,26,133.0
32740,2012,13,22,29,63.0
32741,2012,13,22,29,103.0
32742,2012,13,22,29,96.0
32743,2011,13,22,29,45.0


# L1 regularization (L1正則化)
* 一変量法は、選択決定を行うときに、一度に**1つの特徴のみ**を考慮します。
* その代わりに、L1正則化を持つ線形モデルにそれらを含めることで、すべての特徴を使用して選択を行うことが出来ます。
* このタイプの正則化(Lassoと呼ばれることがある）は、係数の二乗にペナルティをかけるL2(Ridhe)回帰と比較して、
    * **係数の絶対的な大きさに**ペナルティをかけます
* 正則化の強さを大きくすると、目標を予測するうえで、重要度の低い特徴は0に設定されます。
* これにより、正則化パラメータを調整することで、特徴の選択を行うことが出来ます。
* パラメータは、hold-out setで最高の性能を見つけることによって選択するか、あるいは、
* 保持する特徴の数を事前に決めておきます

<br>

* 回帰問題では、
    * 分類に、**sklearn.linear_model.Lassoや、sklearn.linear_model.LogisticRegression**を使うことが出来ます。
    * これらは、非ゼロの係数を選択するために、
        * **sklearn.feature_selection.SelectFromModel**と一緒に使うことが出来ます。
    * それ以外の場合、コードは1変量検定と似ています。

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

train, valid, _ = get_data_splits(baseline_data)

X,y = train[train.columns.drop("outcome")], train['outcome']

logistic = LogisticRegression(C=1, penalty = "l1", solver='liblinear', random_state = 7).fit(X,y)

model = SelectFromModel(logistic, prefit = True) # なんだよこれ
#  Either fit the model before transform or set "prefit=True" while passing the fitted estimator to the constructor.

X_new = model.transform(X)
X_new

array([[1.000e+03, 1.200e+01, 1.100e+01, ..., 1.119e+03, 1.700e+01,
        1.600e+02],
       [3.000e+04, 4.000e+00, 2.000e+00, ..., 9.640e+02, 2.900e+01,
        9.800e+01],
       [4.500e+04, 0.000e+00, 1.200e+01, ..., 9.640e+02, 2.900e+01,
        8.100e+01],
       ...,
       [5.000e+02, 2.300e+01, 2.000e+00, ..., 1.318e+03, 2.900e+01,
        9.600e+01],
       [8.000e+04, 1.900e+01, 2.400e+01, ..., 1.174e+03, 2.900e+01,
        1.530e+02],
       [1.750e+04, 2.200e+01, 2.300e+01, ..., 1.174e+03, 2.900e+01,
        8.400e+01]])

* 選択された列を取得できるように、これらをDataFrameに変換します

In [15]:
selected_features = pd.DataFrame(model.inverse_transform(X_new),
                                 index = X.index,
                                 columns = X.columns)


selected_columns = selected_features.columns[selected_features.var() != 0]

In [19]:
len(selected_columns)

12

In [20]:
selected_columns

Index(['goal', 'hour', 'day', 'month', 'year', 'category', 'currency',
       'country', 'category_currency', 'category_country', 'currency_country',
       'count_7_days'],
      dtype='object')

In [21]:
len(baseline_data.columns)

14

* このL1パラメータのC=1のこのケースでは、time_since_last_projectの絡むを落としています。
* 一般的に、L1正則化を用いた特徴選択は、一変量検定よりも強力ですが、多くのデータと多くの特徴がある場合には、
* 非常に多くなることもあります。
* 一変量決定は、大規模なデータセットでは、
* **はるかに高速になりますが、パフォーマンスも悪くなる可能性が高いです**

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, metrics
import lightgbm as lgb

import os

clicks = pd.read_parquet('../input/feature-engineering-data/baseline_data.pqt')
data_files = ['count_encodings.pqt',
              'catboost_encodings.pqt',
              'interactions.pqt',
              'past_6hr_events.pqt',
              'downloads.pqt',
              'time_deltas.pqt',
              'svd_encodings.pqt']
data_root = '../input/feature-engineering-data'
for file in data_files:
    features = pd.read_parquet(os.path.join(data_root, file))
    clicks = clicks.join(features)

def get_data_splits(dataframe, valid_fraction=0.1):

    dataframe = dataframe.sort_values('click_time')
    valid_rows = int(len(dataframe) * valid_fraction)
    train = dataframe[:-valid_rows * 2]
    # valid size == test size, last two sections of the data
    valid = dataframe[-valid_rows * 2:-valid_rows]
    test = dataframe[-valid_rows:]
    
    return train, valid, test

def train_model(train, valid, test=None, feature_cols=None):
    if feature_cols is None:
        feature_cols = train.columns.drop(['click_time', 'attributed_time',
                                           'is_attributed'])
    dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
    dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
    
    param = {'num_leaves': 64, 'objective': 'binary', 
             'metric': 'auc', 'seed': 7}
    num_round = 1000
    print("Training model!")
    bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], 
                    early_stopping_rounds=20, verbose_eval=False)
    
    valid_pred = bst.predict(valid[feature_cols])
    valid_score = metrics.roc_auc_score(valid['is_attributed'], valid_pred)
    print(f"Validation AUC score: {valid_score}")
    
    if test is not None: 
        test_pred = bst.predict(test[feature_cols])
        test_score = metrics.roc_auc_score(test['is_attributed'], test_pred)
        return bst, valid_score, test_score
    else:
        return bst, valid_score

# これまでに作ったすべての機能のベースラインスコアを見てみましょう


In [ ]:
train, valid, test = get_data_splits(clicks)
_, baseline_score = train_model(train, valid)

## 特徴量選択にどのデータを使うべきか？
* 特徴選択の方法の多くは、データセットから統計量を計算する必要があるので、
* 特徴選択には、すべてのデータを使うべきでしょうか？
* これで、予測に使用している91の特徴が分かりました。
* これらすべての特徴から、モデルがデータにオーバーフィットしている可能性があります。
* いくつかの特徴を削除することで、オーバーフィッティングを減らすことが出来るかもしれません。
* もちろん、モデルのパフォは減るかもしれんけど、少なくとも、性能をあまり落とさずにモデルをより小さく、より早くすることが出来ます。

* 特徴選択にvalid, testを含めると、leakageの原因になります。
    * 訓練セットのみで特徴選択を実行し、その結果を使用して、
    * 検証セットとテストセットから特徴を選択したいと思うでしょう


### 一変量特徴選択
* SelectKBestをf_classifスコアリング関数を使って、データ中の91個の特徴から、40個の特徴を選択します
* 

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
feature_cols = clicks.columns.drop(['click_time', 'attributed_time', 'is_attributed'])
train, valid, test = get_data_splits(clicks)

# Create the selector, keeping 40 features
selector = SelectKBest(f_classif, k = 40)

# Use the selector to retrieve the best features
X_new = selector.fit_transform(train[feature_cols], train['is_attributed'])

# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(selector.inverse_transform(X_new),
                                index = train.index,
                                columns = feature_cols)

# Find the columns that were dropped
dropped_columns = selected_features.columns[selected_features.var() == 0]


In [ ]:
_ = train_model(train.drop(dropped_columns, axis=1), 
                valid.drop(dropped_columns, axis=1))

## Kの最適値
* この方法であれば、最高のKの特徴を選ぶことが出来ますが、
* やはり自分たちでKを選ばなければなりません。Kの最高の値はどうやって見つけるのでしょうか？
    * 最高の機能を維持するためには小さくしたいが、モデルのパフォーマンスを低下させるほど小さくはしたくないということです。
* Kの最適値を見つけるには、Kの値を増やしながら複数のモデルを適合させ、
    * **検証スコアがある閾値またはほかの基準以上**の最小のKを選択することが出来ます
    * これを行う良い方法は、**Kの値をループして、各反復のバリデーション・スコアを記録することです。**

## 特徴選択にL1正則化を使用する
* 保持する特徴のリストを返す関数**select_features_l1**を実装します。
* 特徴を選択するために、L1ペナを課した**LogisticRegression分類器モデル**を使用します。
* モデルには、
    * random_state = 7に,
    * the regularization parameter to 0.1
    * the solver to **liblinear**.
* モデルをフィットし、SelectFromModelを使用して選択された特徴を持つモデルを返します。
* チェックコードは、データせっとのサンプルで関数を実行し、より即時のフィードバックを提供します。

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

def select_features_l1(X, y):
    """Return selected features using logistic regression with an L1 penalty."""
    logistic  = LogisticRegression(C = 0.1, penalty="l1", random_state = 7).fit(X,y)
    model = SelectFromModel(logistic, prefit = True)
    
    X_new = model.transform(X)
    selected_features = pd.DataFrame(model.inverse_transform(X_new),
                                    index = X.index,
                                    columns = X.columns)
    cols_to_keep = selected_features.columns[selected_features.var() != 0]

    return cols_to_keep


In [ ]:
n_samples = 10000
X, y = train[feature_cols][:n_samples], train['is_attributed'][:n_samples]
selected = select_features_l1(X, y)

dropped_columns = feature_cols.drop(selected)
_ = train_model(train.drop(dropped_columns, axis=1), 
                valid.drop(dropped_columns, axis=1))

* なんで削ってんだ？こいつ...**削ったほうが性能が高い**

## 木による特徴の選択
* 木ベースのモデルを使っているので、特徴選択のために別の木のベースのモデルを使ったほうが、
* よりよい結果が得られるかもしれません。
* 木の分類器を使って特徴を選択する場合、何か違うことをしますか？

* RandomForestClassifierや、
* ExtraTreesClassifierのようなものを使って特徴のインポートを見つけることが出来ます。
* SelectFromModelは、特徴の重要さを使用して最適な特徴量を見つけることが出来ます。


## L1正則化を用いたトップK特徴量
* ここでは、正則化パラメータ C = 0.1を設定しているために、
* いくつかの特徴量が削除されています。
* しかし、Cを設定することで、保持する特徴の数を選択することが出来ません。
* L1正則化を用いて、上位K個の重要な特徴を保持するにはどうしたらよいでしょうか

* L1正則化を用いてある数の特徴を選択するには、
* 目的の数の特徴を残す正則化パラメータを見つける必要があります。
* これを行うには、正則化パラメータが低い物から高い物まで、異なる正則化パラメータを持つモデルを反復処理して、K個の特徴を残すものを選択します。
* scikit-learnモデルではCは正則化の強さの逆数であることに注意して